In [1]:
import pandas as pd 

C:\Users\SAMSUNG\AppData\Local\Temp\ipykernel_7812\217833862.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
from pykrx import stock
stock.get_market_cap("20190101", "20190102", "005930")

""


### pbr 구하기
- '자산,자본,부채,매출.csv' 파일 읽기
- 연결데이터가 존재할 경우 연결데이터를 사용하고, 연결데이터 없을경우 개별 데이터 사용하기
- 시가총액 구하기 
- pbr = 시가총액/자산
- 배당성향 = 배당금/순이익 

In [3]:
df=pd.read_csv('./data/자산,자본,부채,매출.csv',encoding='cp949')

#### 개별, 연결 합치기

- 연결이 붙은 열을 추출해 개별 df로 만들기 (filtered_df)
- 기존의 df에서 연결이 붙은 열을 드랍하기
- 두 데이터프레임의 열 이름을 깔끔하게 정리하기 (for문 사용, 제거할 내용을 리스트화하기)
- 기존 df와 filtered_df를 합치기. 이 때, filtered_df가 연결값이므로 우선시 되어야함 

In [ ]:
filtered_df = df.filter(regex='연결')
filtered_df[['회사명','거래소코드','회계년도']] =df[['회사명','거래소코드','회계년도']]

#연결 이름이 붙은 열 제거하기 
df=df.drop(df.filter(regex='연결').columns,axis=1)

In [18]:
strs = ['(*)', '(IFRS연결)', '(천원)', '(IFRS)', '[제조]',
         '*','(-)',('결손금'),'(수익)','(보고서기재)','()','(정상)']
#위에 포함된 모든 문자열을 삭제시키기
for s in strs:
    filtered_df.columns = filtered_df.columns.str.replace(s, '')
    df.columns = df.columns.str.replace(s, '')

## 공백제거하기
filtered_df.columns = filtered_df.columns.str.strip()
filtered_df.columns

df.columns = df.columns.str.strip()
df.columns

Index(['회사명', '거래소코드', '회계년도', '자산', '자본', '부채', '매출액'], dtype='object')

In [22]:
#df의 값이 우선사용되고 값이 없을경우에 df2값이 사용됨 
#df=df.combine_first(df2)
df=filtered_df.combine_first(df)

In [25]:
filtered_df.head(6)

,자산,자본,부채,매출액,회사명,거래소코드,회계년도
0,114379880.0,103981846.0,10398033.0,49864782.0,(주)CMG제약,58820,2018/12
1,192243166.0,180158283.0,12084883.0,58622681.0,(주)CMG제약,58820,2019/12
2,NaN,NaN,NaN,NaN,(주)CMG제약,58820,2020/12
3,NaN,NaN,NaN,NaN,(주)CMG제약,58820,2021/12
4,NaN,NaN,NaN,NaN,(주)CMG제약,58820,2022/12
5,242098205.0,190232688.0,51865516.0,93906675.0,(주)CMG제약,58820,2023/12


#### 시가총액 구하기 
- df의 거래소코드에 제로패딩하기 
- 18~23년의 6년도 데이터 존재. 이때 연말의 시가총액만 구해서 '시가총액'열에다가 값 넣기

In [28]:
df['거래소코드'] = df['거래소코드'].astype(str).str.zfill(6)

#### for문 만들기 위해 회사 한개로 연습해보기

- 회계년도 열을 데이트타임 형식으로 변경하기 
- '회계년도말일' 이라는 열을 만들어 strftime함수를 사용해  회계년도 열의 day값을 31로 변경하기 

In [37]:
df_example=df.head(6)
df_example['회계년도'] = pd.to_datetime(df_example['회계년도'])
df_example['회계년도말일'] = df_example['회계년도'].dt.strftime('%Y-%m-31')
df_example

C:\Users\djssu\AppData\Local\Temp\ipykernel_21076\1829793704.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_example['회계년도'] = pd.to_datetime(df_example['회계년도'])
C:\Users\djssu\AppData\Local\Temp\ipykernel_21076\1829793704.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_example['회계년도'] = pd.to_datetime(df_example['회계년도'])
C:\Users\djssu\AppData\Local\Temp\ipykernel_21076\1829793704.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

,거래소코드,매출액,부채,자본,자산,회계년도,회사명,회계년도말일
0,058820,49864782.0,10398033.0,103981846.0,114379880.0,2018-12-01,(주)CMG제약,2018-12-31
1,058820,58622681.0,12084883.0,180158283.0,192243166.0,2019-12-01,(주)CMG제약,2019-12-31
2,058820,64459891.0,14224578.0,181131004.0,195355583.0,2020-12-01,(주)CMG제약,2020-12-31
3,058820,69475496.0,14472099.0,184702494.0,199174593.0,2021-12-01,(주)CMG제약,2021-12-31
4,058820,82197410.0,28307361.0,184953015.0,213260376.0,2022-12-01,(주)CMG제약,2022-12-31
5,058820,93906675.0,51865516.0,190232688.0,242098205.0,2023-12-01,(주)CMG제약,2023-12-31


### 만들어낸 회계년도 말일을 통해 연말의 시가총액 1개만 구하기

- 저장할 빈 리스트를 우선 만들어야 할것
- 시가총액 코드를 보면서 내가 구한 걸 넣어야함. 
- 이때, 최종값 1개만 넣으면 되므로 특정함수를 사용해야할것임 

In [51]:
# stock.get_market_cap("20190101", "20190102", "005930") << 시가총액 코드 
marcap=[]

for i in range(len(df_example)):
    marcap.append(stock.get_market_cap(df_example['회계년도'][i],
                         df_example['회계년도말일'][i],
                         df_example['거래소코드'][i]).tail(1)['시가총액'][0])

C:\Users\djssu\AppData\Local\Temp\ipykernel_21076\2485771387.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  marcap.append(stock.get_market_cap(df_example['회계년도'][i],
C:\Users\djssu\AppData\Local\Temp\ipykernel_21076\2485771387.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  marcap.append(stock.get_market_cap(df_example['회계년도'][i],
C:\Users\djssu\AppData\Local\Temp\ipykernel_21076\2485771387.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use

In [53]:
df_example['시가총액']=marcap

C:\Users\djssu\AppData\Local\Temp\ipykernel_21076\4027275905.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_example['시가총액']=marcap


In [54]:
df_example

,거래소코드,매출액,부채,자본,자산,회계년도,회사명,회계년도말일,시가총액
0,058820,49864782.0,10398033.0,103981846.0,114379880.0,2018-12-01,(주)CMG제약,2018-12-31,461137478340
1,058820,58622681.0,12084883.0,180158283.0,192243166.0,2019-12-01,(주)CMG제약,2019-12-31,590292037000
2,058820,64459891.0,14224578.0,181131004.0,195355583.0,2020-12-01,(주)CMG제약,2020-12-31,802797170320
3,058820,69475496.0,14472099.0,184702494.0,199174593.0,2021-12-01,(주)CMG제약,2021-12-31,532651755740
4,058820,82197410.0,28307361.0,184953015.0,213260376.0,2022-12-01,(주)CMG제약,2022-12-31,298618324600
5,058820,93906675.0,51865516.0,190232688.0,242098205.0,2023-12-01,(주)CMG제약,2023-12-31,334730308040
